# Model Comparison


## Imports and Config

In [1]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Subset, random_split
from tqdm.notebook import tqdm
import numpy as np
from torchinfo import summary

In [3]:
from data.dataset import LeakAnomalyDetectionDataset
from models.classifiers import RNNClassfier
from models.autoencoders import SimpleAutoencoder, ScheduledSamplingAutoencoder, RNNAutoencoder, criterion as autoencoder_criterion
from config import *
import utils

In [4]:
utils.set_device()
utils.set_seed()

Using device = cpu
